In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import csv
import json

In [2]:
linkedin_url = "https://www.linkedin.com/jobs/search"

In [3]:
query_string = {
    "keywords": "Desenvolvedor Junior", #Title
    "location": "Brazil", # Country 
    "trk": "public_jobs_jobs-search-bar_search-submit" #Linkedin Tracking
}

In [4]:
response = rq.get(linkedin_url, params=query_string) # Accessing the page and verifying the status
response.status_code

200

In [5]:
content = bs(response.content, 'html.parser') # Using BeautifulSoup to convert the HTML page

In [6]:
vacancy_list = content.select(".jobs-search__results-list li") # Getting the list of vacancies in the page

In [7]:
vacancies_list = []
for vacancy in vacancy_list:
    title = vacancy.select(".base-search-card__title")[0].text.strip() # Title
    # obs.: we could use find() instead of select() in some cases.
    
    company = vacancy.select(".base-search-card__subtitle")[0].text.strip() # Subtitle/Company
    
    location = vacancy.select(".job-search-card__location")[0].text.strip() # Location
    
    url = vacancy.select(".base-card__full-link")[0].get('href') # URL
    
    date = vacancy.select("time")[0].get("datetime") # Date
    
    # Putting the informations in a dict, so we can add this dict on the Vacancies List
    info = {
        "Title": title,
        "Company": company,
        "Location": location,
        "URL": url,
        "Date": date
    }
    vacancies_list.append(info)

In [8]:
vacancies_list_json = json.dumps(vacancies_list) # Putting the list in a text format

In [9]:
with open("vacancies.json", "w") as json_archive:
    json_archive.write(vacancies_list_json) # This way we write the list on a .json archive

In [10]:
# Putting the list in a .csv file
with open('vacancies.csv', mode='w') as csv_file:
    header = ['Title', 'Company', 'Location', 'URL', 'Date']
    generator_csv = csv.DictWriter(csv_file, fieldnames=header)
    generator_csv.writeheader()
    generator_csv.writerows(vacancies_list)